In [299]:
import json
import urllib
import pandas as pd
import datetime

# Historical Weather data

In [300]:
end_date=datetime.datetime.strptime("2023-01-14","%Y-%m-%d")
start_date=end_date+datetime.timedelta(days=-40)


In [301]:
base_url="https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline"
weather_api="F5B6R3DHJAYYTDG9XVLMMHRBR"
lat=1.3521
lng=103.8198
location=f"{lat},{lng}"
unitGroup="metric"
Include="days"
start=start_date.strftime("%Y-%m-%d")
end=end_date.strftime("%Y-%m-%d")

In [302]:
history_query=f"{base_url}/{location}/{start}/{end}?&unitGroup={unitGroup}&include={Include}&key={weather_api}"

In [303]:
history_query

'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/1.3521,103.8198/2022-12-05/2023-01-14?&unitGroup=metric&include=days&key=F5B6R3DHJAYYTDG9XVLMMHRBR'

In [304]:
weather_json=urllib.request.urlopen(history_query)

In [305]:
weather_data=json.loads(weather_json.read())

In [306]:
weather_training_data=pd.DataFrame(weather_data["days"])

In [307]:
weather_training_data.columns

Index(['datetime', 'datetimeEpoch', 'tempmax', 'tempmin', 'temp',
       'feelslikemax', 'feelslikemin', 'feelslike', 'dew', 'humidity',
       'precip', 'precipprob', 'precipcover', 'preciptype', 'snow',
       'snowdepth', 'windgust', 'windspeed', 'winddir', 'pressure',
       'cloudcover', 'visibility', 'solarradiation', 'solarenergy', 'uvindex',
       'severerisk', 'sunrise', 'sunriseEpoch', 'sunset', 'sunsetEpoch',
       'moonphase', 'conditions', 'description', 'icon', 'stations', 'source'],
      dtype='object')

In [308]:
weather_training_data["datetime"]=weather_training_data["datetime"].apply(lambda x:datetime.datetime.strptime(x,"%Y-%m-%d"))

# Forcast function

In [309]:
today=datetime.date.today()
def forcast_raw(today):
    base_url="https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline"
    weather_api="F5B6R3DHJAYYTDG9XVLMMHRBR"
    lat=1.3521
    lng=103.8198
    location=f"{lat},{lng}"
    unitGroup="metric"
    Include="days"
    start_date=today
    end_date=start_date+datetime.timedelta(days=6)
    start=start_date.strftime("%Y-%m-%d")
    end=end_date.strftime("%Y-%m-%d")
    forcast_query=f"{base_url}/{location}/{start}/{end}?&unitGroup={unitGroup}&include={Include}&key={weather_api}"
    forcast_json=urllib.request.urlopen(forcast_query)
    forcast_data=json.loads(forcast_json.read())
    forcast_data=pd.DataFrame(forcast_data["days"])
    forcast_data["datetime"]=forcast_data["datetime"].apply(lambda x:datetime.datetime.strptime(x,"%Y-%m-%d"))
    return forcast_data

In [310]:
forcast_data=forcast_raw(today)

In [311]:
forcast_data.columns

Index(['datetime', 'datetimeEpoch', 'tempmax', 'tempmin', 'temp',
       'feelslikemax', 'feelslikemin', 'feelslike', 'dew', 'humidity',
       'precip', 'precipprob', 'precipcover', 'preciptype', 'snow',
       'snowdepth', 'windgust', 'windspeed', 'winddir', 'pressure',
       'cloudcover', 'visibility', 'solarradiation', 'solarenergy', 'uvindex',
       'severerisk', 'sunrise', 'sunriseEpoch', 'sunset', 'sunsetEpoch',
       'moonphase', 'conditions', 'description', 'icon', 'stations', 'source'],
      dtype='object')

# Historical Air quality data

In [312]:
aqi_data=pd.read_csv("aqi_training_data.csv",index_col=0)

In [313]:
aqi_data.head()

,datetime,aqi
0,17/12/2022,29
1,18/12/2022,33
2,19/12/2022,29
3,20/12/2022,27
4,21/12/2022,31


# change datetime to datetime datatype

In [314]:
#change the data type of 
aqi_data["datetime"]=aqi_data["datetime"].apply(lambda x:datetime.datetime.strptime(x,"%d/%m/%Y"))

In [315]:
aqi_data["datetime"].dtype

dtype('<M8[ns]')

# Combine weather and aqi data

In [446]:
combined_training_data=weather_training_data.merge(aqi_data,on="datetime", how="outer")

In [447]:
combined_training_data.head(1)

,datetime,datetimeEpoch,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,sunriseEpoch,sunset,sunsetEpoch,moonphase,conditions,description,icon,stations,source,aqi
0,2022-12-05,1.670170e+09,30.4,25.1,26.5,35.7,25.1,27.6,24.3,88.0,...,1.670194e+09,18:56:37,1.670238e+09,0.47,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,"[48698099999, 48679099999, WSAP, 48694099999, ...",obs,NaN


In [448]:
combined_training_data=combined_training_data.sort_values(by="datetime")

# Shift aqi columns to include 7 historical aqi for each row

In [449]:
combined_training_data["aqi_1"]=combined_training_data["aqi"].shift(1)
combined_training_data["aqi_2"]=combined_training_data["aqi"].shift(2)
combined_training_data["aqi_3"]=combined_training_data["aqi"].shift(3)
combined_training_data["aqi_4"]=combined_training_data["aqi"].shift(4)
combined_training_data["aqi_5"]=combined_training_data["aqi"].shift(5)
combined_training_data["aqi_6"]=combined_training_data["aqi"].shift(6)
combined_training_data["aqi_7"]=combined_training_data["aqi"].shift(7)

In [450]:
combined_training_data.tail()

,datetime,datetimeEpoch,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,stations,source,aqi,aqi_1,aqi_2,aqi_3,aqi_4,aqi_5,aqi_6,aqi_7
37,2023-01-11,1.673366e+09,29.4,25.0,26.0,33.4,25.0,26.6,23.4,85.7,...,"[48698099999, 48679099999, WSAP, 48694099999, ...",obs,59.0,72.0,54.0,41.0,38.0,46.0,46.0,26.0
38,2023-01-12,1.673453e+09,32.4,24.0,27.6,36.8,24.0,29.7,22.8,76.6,...,"[48698099999, 48679099999, WSAP, 48694099999, ...",obs,57.0,59.0,72.0,54.0,41.0,38.0,46.0,46.0
39,2023-01-13,1.673539e+09,31.2,15.3,26.0,36.3,15.3,27.1,23.2,85.2,...,"[48698099999, EUCDE37, 48679099999, WSAP, 4869...",obs,54.0,57.0,59.0,72.0,54.0,41.0,38.0,46.0
40,2023-01-14,1.673626e+09,30.9,11.2,25.7,36.4,11.2,27.3,22.8,84.8,...,"[EUCDE37, WSAP, WSSS, WIDD, WSSL, WMKJ]",obs,65.0,54.0,57.0,59.0,72.0,54.0,41.0,38.0
41,2023-01-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,65.0,65.0,54.0,57.0,59.0,72.0,54.0,41.0


# Drop unnecessary columns

In [451]:
combined_training_data=combined_training_data.dropna(subset=[f"aqi_{i}" for i in range(1,8)],how="any",axis="rows")

In [452]:
combined_training_data.tail(1)

,datetime,datetimeEpoch,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,stations,source,aqi,aqi_1,aqi_2,aqi_3,aqi_4,aqi_5,aqi_6,aqi_7
41,2023-01-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,65.0,65.0,54.0,57.0,59.0,72.0,54.0,41.0


In [453]:
combined_training_data=combined_training_data.reset_index()

In [454]:
#drop today's data
combined_training_data=combined_training_data.drop([len(combined_training_data)-1],axis=0)

In [455]:
columns_to_drop=["index","datetime","datetimeEpoch","preciptype","snowdepth","sunrise","sunset","icon","stations","source","description","conditions"]

In [456]:
combined_training_data=combined_training_data.drop(columns_to_drop,axis=1)

In [457]:
y_train=combined_training_data.aqi

In [458]:
combined_training_data=combined_training_data.drop(["aqi"],axis=1)

In [459]:
combined_training_data.tail(1)

,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,precipprob,...,sunriseEpoch,sunsetEpoch,moonphase,aqi_1,aqi_2,aqi_3,aqi_4,aqi_5,aqi_6,aqi_7
21,30.9,11.2,25.7,36.4,11.2,27.3,22.8,84.8,12.3,100.0,...,1.673652e+09,1.673695e+09,0.72,54.0,57.0,59.0,72.0,54.0,41.0,38.0


# Train model

In [460]:
from xgboost import XGBRegressor

In [461]:
x_train=combined_training_data

In [462]:
model=XGBRegressor()
model.fit(x_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [463]:
from sklearn.metrics import mean_squared_error as MSE

In [464]:
MSE(y_train,model.predict(x_train))

1.9686128573746167e-07

In [465]:
model.predict(x_train)

array([56.999176, 45.00005 , 46.00028 , 51.00036 , 55.00015 , 59.999874,
       60.99955 , 56.999836, 58.99983 , 60.      , 31.001062, 26.000216,
       46.000237, 46.000248, 37.99987 , 41.000336, 53.999718, 71.99881 ,
       59.00005 , 57.000233, 54.000317, 64.99973 ], dtype=float32)

In [466]:
y_train

0     57.0
1     45.0
2     46.0
3     51.0
4     55.0
5     60.0
6     61.0
7     57.0
8     59.0
9     60.0
10    31.0
11    26.0
12    46.0
13    46.0
14    38.0
15    41.0
16    54.0
17    72.0
18    59.0
19    57.0
20    54.0
21    65.0
Name: aqi, dtype: float64

In [511]:
def feature_engineering(data,prev_aqi):
    n=len(data)
    
    prev_aqi=pd.concat([prev_aqi for i in range(n)],axis=0)
    prev_aqi=prev_aqi.reset_index()
    prev_aqi=prev_aqi.drop(["index"],axis=1)
    print(prev_aqi)
    comb=pd.concat([data,prev_aqi],axis=1)
    columns_to_drop=["datetime","datetimeEpoch","preciptype","snowdepth","sunrise","sunset","icon","stations","source","description","conditions"]
    comb=comb.drop(columns_to_drop,axis=1)
    return comb

In [499]:
x=pd.DataFrame({"A":[1]})

In [500]:
pd.concat([x,x],axis=1)

,A,A
0,1,1


In [501]:
latest_aqi=x_train[[f"aqi_{i}" for i in range(1,8)]].tail(1)

In [502]:
pd.concat([latest_aqi,latest_aqi])

,aqi_1,aqi_2,aqi_3,aqi_4,aqi_5,aqi_6,aqi_7
21,54.0,57.0,59.0,72.0,54.0,41.0,38.0
21,54.0,57.0,59.0,72.0,54.0,41.0,38.0


In [503]:
forcast_data

,datetime,datetimeEpoch,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,sunrise,sunriseEpoch,sunset,sunsetEpoch,moonphase,conditions,description,icon,stations,source
0,2023-01-15,1673712000,31.3,24.9,27.3,36.2,24.9,29.7,23.9,82.0,...,07:12:30,1673737950,19:15:27,1673781327,0.78,"Rain, Partially cloudy",Partly cloudy throughout the day with storms p...,rain,"[WSAP, WSSS, WIDD, WSSL, WMKJ]",comb
1,2023-01-16,1673798400,29.2,25.6,27.0,32.4,25.6,28.3,22.8,78.1,...,07:12:50,1673824370,19:15:50,1673867750,0.83,"Rain, Partially cloudy",Partly cloudy throughout the day with storms p...,rain,None,fcst
2,2023-01-17,1673884800,28.3,25.1,26.6,31.3,25.1,27.7,22.6,78.9,...,07:13:09,1673910789,19:16:11,1673954171,0.88,"Rain, Overcast",Cloudy skies throughout the day with a chance ...,rain,None,fcst
3,2023-01-18,1673971200,28.3,24.9,26.4,31.3,24.9,27.3,22.5,79.7,...,07:13:28,1673997208,19:16:32,1674040592,0.93,"Rain, Overcast",Cloudy skies throughout the day with a chance ...,rain,None,fcst
4,2023-01-19,1674057600,29.0,24.9,26.9,32.6,24.9,28.5,22.7,78.4,...,07:13:46,1674083626,19:16:52,1674127012,0.96,"Rain, Overcast",Cloudy skies throughout the day with storms po...,rain,None,fcst
5,2023-01-20,1674144000,28.7,24.8,26.8,31.9,24.8,28.3,23.0,79.9,...,07:14:03,1674170043,19:17:11,1674213431,0.99,"Rain, Overcast",Cloudy skies throughout the day with storms po...,rain,None,fcst
6,2023-01-21,1674230400,28.0,25.2,26.4,30.4,25.2,27.2,22.6,80.1,...,07:14:20,1674256460,19:17:30,1674299850,1.00,"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...,rain,None,fcst


In [512]:
forcast_input=feature_engineering(forcast_data,latest_aqi)

   aqi_1  aqi_2  aqi_3  aqi_4  aqi_5  aqi_6  aqi_7
0   54.0   57.0   59.0   72.0   54.0   41.0   38.0
1   54.0   57.0   59.0   72.0   54.0   41.0   38.0
2   54.0   57.0   59.0   72.0   54.0   41.0   38.0
3   54.0   57.0   59.0   72.0   54.0   41.0   38.0
4   54.0   57.0   59.0   72.0   54.0   41.0   38.0
5   54.0   57.0   59.0   72.0   54.0   41.0   38.0
6   54.0   57.0   59.0   72.0   54.0   41.0   38.0


In [513]:
def forcast_aqi(x,model):
    return model.predict(x)

In [514]:
forcast_aqi(forcast_input,model)

array([62.166447, 55.227192, 41.453857, 41.454544, 41.454453, 41.0452  ,
       41.06672 ], dtype=float32)

In [497]:
forcast_input

,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,precipprob,...,sunsetEpoch,moonphase,index,aqi_1,aqi_2,aqi_3,aqi_4,aqi_5,aqi_6,aqi_7
0,31.3,24.9,27.3,36.2,24.9,29.7,23.9,82.0,3.2,100.0,...,1673781327,0.78,21,54.0,57.0,59.0,72.0,54.0,41.0,38.0
1,29.2,25.6,27.0,32.4,25.6,28.3,22.8,78.1,19.0,100.0,...,1673867750,0.83,21,54.0,57.0,59.0,72.0,54.0,41.0,38.0
2,28.3,25.1,26.6,31.3,25.1,27.7,22.6,78.9,17.0,85.7,...,1673954171,0.88,21,54.0,57.0,59.0,72.0,54.0,41.0,38.0
3,28.3,24.9,26.4,31.3,24.9,27.3,22.5,79.7,3.7,71.4,...,1674040592,0.93,21,54.0,57.0,59.0,72.0,54.0,41.0,38.0
4,29.0,24.9,26.9,32.6,24.9,28.5,22.7,78.4,8.5,57.1,...,1674127012,0.96,21,54.0,57.0,59.0,72.0,54.0,41.0,38.0
5,28.7,24.8,26.8,31.9,24.8,28.3,23.0,79.9,18.3,85.7,...,1674213431,0.99,21,54.0,57.0,59.0,72.0,54.0,41.0,38.0
6,28.0,25.2,26.4,30.4,25.2,27.2,22.6,80.1,16.0,90.5,...,1674299850,1.00,21,54.0,57.0,59.0,72.0,54.0,41.0,38.0
